In [1]:
import pandas as pd 
import os

In [2]:
geoDF = pd.read_csv('./geodata.csv', index_col = 0)

In [3]:
geoDF.head()

,Site Name,Location,Coordinates,Landscape,Climate,Reference,Site PI,Data Available
Site ID,,,,,,,,
1,Bermuda Island,"Bermuda Island, England","32°15’N, 64°52’W",Coast,Dwa,Steen-Larsen et al. (2015),Hans Christian Steen-Larsen,In prep.
2,Beijing,"Beijing, China","40°0’N, 116°3’E",Urban,Dwa,Wen et al. (2010),Xuhui Lee and Xuefa Wen,Yes
3,Borden,"Borden, Canada","44°9′N, 79°6′W",forest,Dfb,Santos et al. (2012),Xuhui Lee,Yes
4,Camargue,"Vaccarès lagoon, France","43°35’N, 4°28’E",Wetland,Csb,Delattre et al. (2015),Christine Vallet-Coulomb,Yes
5,Dome C,"Concordia, South Pole","75°06′S, 123°23′E",Ice Sheet,-,Casado et al (2016),Mathieu Casado,Yes


In [20]:
isodfs = []
isopath = "./isotope-data/"
for filename in os.listdir(isopath):
    print(isopath + filename)
    df = pd.read_csv(isopath + filename)
    if 'Delta_18O' in df.columns and 'Delta_D' in df.columns:
        df = df[df['Delta_18O'] != -9999]
        df = df[df['Delta_D'] != -9999]
        
        if(len(df.index) > 0):
            print(filename + " has nontrivial data")
            isodfs.append(df)
    

./isotope-data/JP-Nagoya.csv
./isotope-data/Ship-RARAAVIS.csv
Ship-RARAAVIS.csv has nontrivial data
./isotope-data/SP-Teide.csv
SP-Teide.csv has nontrivial data
./isotope-data/Ship-PIRATAFR24.csv
Ship-PIRATAFR24.csv has nontrivial data
./isotope-data/US-NewHaven.csv
US-NewHaven.csv has nontrivial data
./isotope-data/CN-Gaoqiao.csv
CN-Gaoqiao.csv has nontrivial data
./isotope-data/AP-Kohnen.csv
AP-Kohnen.csv has nontrivial data
./isotope-data/CH-Rietholzbach.csv
CH-Rietholzbach.csv has nontrivial data
./isotope-data/AP-DomeC.csv
AP-DomeC.csv has nontrivial data
./isotope-data/Ship-JARE55.csv
Ship-JARE55.csv has nontrivial data
./isotope-data/US-Rosemount_G21.csv
./isotope-data/Ship-Bermuda.csv
Ship-Bermuda.csv has nontrivial data
./isotope-data/CN-Beijing.csv
CN-Beijing.csv has nontrivial data
./isotope-data/Ship-ACTIV.csv
Ship-ACTIV.csv has nontrivial data
./isotope-data/RU-Kourovka.csv
RU-Kourovka.csv has nontrivial data
./isotope-data/US-ManitouForest.csv
US-ManitouForest.csv has non

In [30]:
megadf = pd.concat(isodfs).filter(['Delta_18O', 'Delta_D', 'Latitude', 'Longitude', 'TIMESTAMP', 'TIMESTAMP_START'])

In [31]:
megadf.columns

Index(['Delta_18O', 'Delta_D', 'Latitude', 'TIMESTAMP', 'TIMESTAMP_START'], dtype='object')

In [32]:
megadf.head()

,Delta_18O,Delta_D,Latitude,TIMESTAMP,TIMESTAMP_START
0,-14.56,-104.74,5.065,2.014050e+11,NaN
1,-14.23,-102.42,5.018,2.014050e+11,NaN
2,-13.91,-99.90,4.971,2.014050e+11,NaN
3,-13.63,-97.58,4.923,2.014050e+11,NaN
4,-13.45,-96.17,4.875,2.014050e+11,NaN
